In [1]:
#2025-04-28-tutorial

from IPython.display import display, Math


In [2]:
display(Math(r' H =  -J_1 \sum_{i=1}^{L-1} \sigma_i^z \sigma_{i+1}^z  - J_2 \sum_{i=1}^{L-2} \sigma_i^z \sigma_{i+2}^z  - h \sum_{i=1}^{L} \sigma_i^x  - g \sum_{i=1}^{L} \sigma_i^z  - \frac{J_{xy}}{2} (1 + \gamma) \sum_{i=1}^{L-1} (1 - (-1)^i \delta)\, \sigma_i^x \sigma_{i+1}^x  - \frac{J_{xy}}{2} (1 - \gamma) \sum_{i=1}^{L-1} (1 - (-1)^i \delta)\, \sigma_i^y \sigma_{i+1}^y'))

<IPython.core.display.Math object>

In [3]:
display(Math(r'\text{Tutorial}'))

<IPython.core.display.Math object>

In [4]:
display(Math(r'\text{Quantum Ising model (with open boundary conditions)}'))
display(Math(r'H = -J_1 \sum_{\langle i,j \rangle} \sigma_i^z \sigma_j^z - h \sum_i \sigma_i^x'))
display(Math(r'\text{Define Hamiltonian and set the parameters}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [5]:

import time

start = time.time()

import numpy as np
from scipy.sparse import kron, identity, csr_matrix
from scipy.sparse.linalg import eigsh


# Define Pauli matrices
sx = csr_matrix(np.array([[0, 1], [1, 0]], dtype=complex))
sy = csr_matrix(np.array([[0, -1j], [1j, 0]], dtype=complex))
sz = csr_matrix(np.array([[1, 0], [0, -1]], dtype=complex))
id2 = identity(2, format='csr', dtype=complex)

def kron_n(ops):
    """Kronecker product of a list of operators."""
    result = ops[0]
    for op in ops[1:]:
        result = kron(result, op, format='csr')
    return result


def build_hamiltonian(L, J1, J2, h, g, J_xy, delta, gamma):
    H = csr_matrix((2**L, 2**L), dtype=complex)

    # Build -J1 * sum sigma^z_i sigma^z_{i+1}
    for i in range(L-1):
        ops = [id2] * L
        ops[i] = sz
        ops[(i + 1)] = sz
        H -= J1 * kron_n(ops)

    # Build -J2 * sum sigma^z_i sigma^z_{i+2}
    for i in range(L-2):
        ops = [id2] * L
        ops[i] = sz
        ops[(i + 2)] = sz
        H -= J2 * kron_n(ops)

    # Build -h * sum sigma^x_i
    for i in range(L):
        ops = [id2] * L
        ops[i] = sx
        H -= h * kron_n(ops)


    # Build -g * sum sigma^z_i 
    for i in range(L):
        ops = [id2] * L
        ops[i] = sz
        H -= g * kron_n(ops)

    # Build dimerized Hamiltonian
    for i in range(L-1):
        ops = [id2] * L
        ops[i] = sx
        ops[(i + 1)] = sx
        H -= J_xy*(1-(-1)**i*delta)*(1+gamma)/2 * kron_n(ops)

    for i in range(L-1):
        ops = [id2] * L
        ops[i] = sy
        ops[(i + 1)] = sy
        H -= J_xy*(1-(-1)**i*delta)*(1-gamma)/2 * kron_n(ops)

    
    #return the Hamiltonian
    return H


L=4; 
 
J1=1; h=0.3;  
J_xy=0; delta=0.; gamma=0.; J2=0; g=0.; 

H=build_hamiltonian(L, J1, J2, h, g, J_xy, delta, gamma)

# Diagonalize the Hamiltonian
eigenvalues, eigenvectors = eigsh(H, k=2, which='SA')  # Calculate the ground state and the first excited state
#‘SA’ : Smallest (algebraic) eigenvalues.
 

print(f" H0 has L={L}, J1={J1},J2={J2}, h={h}, g={g}, J_xy={J_xy}, delta={delta}, gamma={gamma} ")
#print(eigenvalues)
print("the ground state energy is "+ str(np.min(eigenvalues)))
print("the first excited energy is "+ str(np.max(eigenvalues)))
gap_diag=np.max(eigenvalues)-np.min(eigenvalues)


end = time.time()

print("The energy gap is "+ str(gap_diag))



print(f"\n \n Elapsed time: {end - start:.4f} seconds")

Egs=np.min(eigenvalues)




 H0 has L=4, J1=1,J2=0, h=0.3, g=0.0, J_xy=0, delta=0.0, gamma=0.0 
the ground state energy is -3.1433267964028113
the first excited energy is -3.128581360641489
The energy gap is 0.014745435761322145

 
 Elapsed time: 0.5671 seconds


In [6]:
display(Math(r'\text{Find the ground state energy with an SDP}'))

<IPython.core.display.Math object>

In [7]:
#find the ground state with an SDP


import cvxpy as cp

#define sdp viariables
X = cp.Variable((2**L, 2**L), hermitian=True)  #the quantum state
#define constriants for the quantum state
constraints = []
constraints += [X >> 0]
constraints += [cp.trace(X) == 1]

#define the problem
objective=cp.Minimize(cp.real(cp.trace(H@X)))
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK, verbose=False)#set verbose to true to see details on the calculation

print("problem status = ",prob.status)

Egs_sdp=prob.value
print("The gnd state energy computed with the sdp is", Egs_sdp)


problem status =  optimal
The gnd state energy computed with the sdp is -3.143326832050042
